# Imports

In [ ]:
library(R.matlab)
library(vars)
library("matrixStats")
set.seed(18)
options(warn=-1) 

In [2]:
# download landslide data
mine = readMat('data/mine_data.mat')
mine = mine$data

In [3]:
find_distance = function(x,y){
  return(sqrt((x[1]-y[1])^2+(x[2]-y[2])^2))
}
n.index = c(465,759,769,259,396,1444,90,175,281,507,1662,1038,429,821,1315,63,941,999,929,800,1014,1174,394,1701,825,184,994,480,1555,41)


# Non-stationary

In [ ]:
generate_w = function(n.index,alpha){
  W = matrix(0,nrow = length(n.index),ncol = length(n.index))
  for (i in 1:length(n.index)){
    for (j in 1:length(n.index)) {
      W[i,j] = exp(-alpha*log(find_distance(mine[n.index[i],2:3],mine[n.index[j],2:3])+1))
#       W[i,j] = -(alpha*find_distance(mine[n.index[i],2:3],mine[n.index[j],2:3])^2-400*alpha*find_distance(mine[n.index[i],2:3],mine[n.index[j],2:3]))
    }
    diag(W) = 0
    W[i,] = W[i,] / sum(W[i,]) 
  }
  return(W)
}

In [ ]:
generate_one_step = function(x.t_1, W.t_1,epsilon.t_t){
  n = dim(W.t_1)[1]
  x.t = matrix(0,nrow = n,ncol = 1)
  x.t = W.t_1 %*% x.t_1   + epsilon.t_t  # this is the form of our non-stationary model
  return(x.t)
}

In [ ]:
# The function aims to simulate all data. It iteratively apply generate_one_step function
generate_sim = function(x_1,alphas,T,epsilon){
  x = cbind(x_1)
  n = dim(x_1)[1]
  for(t in 2:T){
    alpha.t_1 = (1 - (t-1)/T)*alphas[1]+(t-1)/T*alphas[2]
#     alpha.t_1 = alphas[t-1] 
    W.t_1 = generate_w(n.index,alpha.t_1)
    tem = matrix(0,nrow = n,ncol = 1)
    tem = generate_one_step(matrix(x[,ncol(x)],nrow = n),W.t_1,epsilon[,t])
    x = cbind(x,tem)
  }
  return(x)
}

In [ ]:
generate_data = function(path, T=500){
    
    epsilon_t = matrix(rnorm(30*500,0,0.1),nrow = 30)
    initial_value = matrix(runif(30,-0.1,0.1),nrow = 30,ncol = 1)
    alpha.bound = runif(1,1,9)*(10^-4)
    alphas = c(alpha.bound, 10*alpha.bound)    
    X = generate_sim(initial_value, alphas, T, epsilon_t)
    write.csv(X, path)
    return(alphas)
    
}

In [ ]:
T = 500
n = 3
alpha.m = matrix(0, nrow=n, ncol=2)
for (i in 0:(n-1)){
    path = paste('data/nst_sim/csv/s',i, '.csv', sep='')
    a = generate_data(path, T)
    alpha.m[i, ] = a
}


In [ ]:
write.csv(alpha.m, 'data/nst_sim/alphas.csv')

In [ ]:
X = read.csv('data/nst_sim/csv/s2.csv')
X = as.matrix(X[, 2:501])

In [ ]:
plot(X[1,])

## Decreasing

In [4]:
generate_w = function(n.index,alpha, d_cutoff){
  W = matrix(0,nrow = length(n.index),ncol = length(n.index))
  for (i in 1:length(n.index)){
    for (j in 1:length(n.index)) {
      W[i,j] = alpha*(-log(find_distance(mine[i,2:3],mine[j,2:3])+1)+log(d_cutoff))
#       W[i,j] = exp(alpha*(-log(find_distance(mine[i,2:3],mine[j,2:3])+1)+log(d_cutoff)))
    }
    diag(W) = 0
#     W[i,] = W[i,] / sum(W[i,]) 
    
  }
  return(W)
}

# function on generate simulated data at one step given previous value, spatial weight matrix, mean function and noise
generate_one_step = function(x.t_1, W.t_1,epsilon.t_t){
  n = dim(W.t_1)[1]
  x.t = matrix(0,nrow = n,ncol = 1)
  x.t = W.t_1 %*% x.t_1  + epsilon.t_t  # this is the form of our non-stationary model
  return(x.t)
}

# The function aims to simulate all data. It iteratively apply generate_one_step function
generate_sim = function(x_1,alpha.bound,T,epsilon,d_cutoff){
  x = cbind(x_1)
  n = dim(x_1)[1]
  for(t in 2:T){
    alpha.t_1 = (1 - (t-1)/T)*alpha.bound[1]+(t-1)/T*alpha.bound[2]
    W.t_1 = generate_w(n.index,alpha.t_1,d_cutoff)
    tem = matrix(0,nrow = n,ncol = 1)
    tem = generate_one_step(matrix(x[,ncol(x)],nrow = n),W.t_1,epsilon[,t])
    x = cbind(x,tem)
  }
  return(x)
}

In [5]:
path = 'data/nst_sim/csv/'
n = 100
alpha.m = matrix(0, nrow=n, ncol=2)

In [6]:
for (i in 0:(n-1)) {
    file_ = paste0('s',as.character(i))
    file_ = paste0(file_,'.csv',seq = "")
    file_name = file.path(path,file_)

    # create epsilon_t
    epsilon_t = matrix(rnorm(30*500,0,1),nrow = 30)
    # initial x_1 at time=1
    initial_value = matrix(runif(30,-0.1,0.1),nrow = 30,ncol = 1)
    alpha.bound = runif(1,1,9)*(10^-4)
    d_cutoff = 170
    x = generate_sim(initial_value,c(alpha.bound,10*alpha.bound),500,epsilon_t,d_cutoff)

    alpha.m[i, ] = c(alpha.bound,10*alpha.bound)

    write.csv(x,file_name)
}

In [ ]:
alpha.m

In [7]:
write.csv(alpha.m, 'data/nst_sim/csv/alphas.csv')

# Stationary

In [ ]:
generate_w = function(n.index,alpha){
  # alpha control the magnitute of shape functions. Now we view alpha*log(d) is shape function.
  W = matrix(0,nrow = length(n.index),ncol = length(n.index))
  for (i in 1:length(n.index)){
    for (j in 1:length(n.index)) {
      W[i,j] = exp(-alpha*log(find_distance(mine[n.index[i],2:3],mine[n.index[j],2:3])+1))
        
    }
    diag(W) = 0
    W[i,] = W[i,] / sum(W[i,])  # flexible depend on results. Adjust accordingly
  }
  return(W)
}


In [ ]:
generate_data = function(path, T=500){
    x1 = runif(30,-0.01,0.01) # 
    X = matrix(0,nrow = 30,ncol = T)
    X[,1] = x1
    # generate W randomly by assign value for alpha.....
    # alpha = random...a value
    W = generate_w(n.index, 1.0)
    epsilon_t = matrix(rnorm(30*T,0,1),nrow=30)
    for (t in 2:T){
      X[,t] = W %*% (matrix(X[,t-1],ncol = 1)) + epsilon_t[,t]
    }
    print(dim(X))
    write.csv(X,path)
    
}


In [ ]:
for (i in 1:100){
    print(i-1)
    path = paste('data/st_sim/csv/s',i-1, '.csv', sep='')
    generate_data(path)
}


In [ ]:
X = read.csv('data/st_sim/csv/s5.csv')
X = as.matrix(X[, 2:501])

In [ ]:
plot(X[1,])

## Decreasing

In [8]:
generate_w = function(n.index,alpha,d_cutoff){
  W = matrix(0,nrow = length(n.index),ncol = length(n.index))
  for (i in 1:length(n.index)){
    for (j in 1:length(n.index)) {
      W[i,j] = alpha*(-log(find_distance(mine[i,2:3],mine[j,2:3])+1)+log(d_cutoff))
#       W[i,j] = alpha * find_distance(mine[i,2:3],mine[j,2:3])^2 -400*alpha* find_distance(mine[i,2:3],mine[j,2:3]) + 40000*alpha
    }
    diag(W) = 0
  }
  return(W)
}

In [9]:
path = 'data/stationary/csv/'

In [10]:
n = 100
alpha.m = matrix(0, nrow=n, ncol=1)

In [11]:
for (i in 0:(n-1)) {
    file_ = paste0('s',as.character(i))
    file_ = paste0(file_,'.csv',seq = "")
    file_name = file.path(path,file_)
    x1 = runif(30,-0.01,0.01) # 
    
    X = matrix(0,nrow = 30,ncol = 500)
    X[,1] = x1

    alpha = runif(1,0.005,0.009) # log
#     alpha = runif(1,0.0000001,0.0000009) # quadratic
    d_cutoff = 170

    W = generate_w(n.index,alpha,d_cutoff)
    epsilon_t = matrix(rnorm(30*500,0,1),nrow = 30)

    for (t in 2:500){
        X[,t] = W %*% (matrix(X[,t-1],ncol = 1))+epsilon_t[,t]
      }

    write.csv(X,file_name)
    alpha.m[i, ] = alpha
}

In [ ]:
alpha.m

In [12]:
write.csv(alpha.m, paste0(path, 'alphas.csv'))